In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR
import os
import geopandas as gpd
from tobler.area_weighted import area_interpolate
from tobler.dasymetric import masked_area_interpolate

from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata
import rasterio

In [ ]:
path = os.getcwd()+ '\..\Dados vetoriais\Pontos amostrais_Piracicaba\Dados_227pts_original_06_03_2021.shp'
pontos_pira = gpd.read_file(path)

In [ ]:
path2 = os.getcwd()+ '\..\Dados vetoriais\Piracicaba\Perimetro_Piracicaba.shp'
perimetro_pira = gpd.read_file(path2)


In [ ]:
xizes = []
ypsolons = []

for geom in perimetro_pira.exterior:
    if(geom == None):
        continue
    xizes.append(geom.coords.xy[0])
    ypsolons.append(geom.coords.xy[1])

xizes = list(np.concatenate(xizes).flat)
ypsolons = list(np.concatenate(ypsolons).flat)

all_dots = [list(xy) for xy in zip(xizes, ypsolons)]

In [ ]:
# split the data into featutes and target variable seperately
variaveis = pontos_pira.iloc[:, [1,2]].values # features set
argila = pontos_pira.iloc[:, 6].values # set of study variable

In [ ]:
argila = argila.reshape(-1,1)

In [ ]:
from sklearn.preprocessing import StandardScaler
StdS_X = StandardScaler()
StdS_y = StandardScaler()
variaveis = StdS_X.fit_transform(variaveis)
argila = StdS_y.fit_transform(argila)

In [ ]:
# import the model
from sklearn.svm import SVR
# create the model object
regressor = SVR(kernel = 'rbf')
# fit the model on the data
regressor.fit(variaveis, argila.ravel())

In [ ]:
A=regressor.predict(StdS_X.transform(np.vstack((xizes, ypsolons)).T))

In [ ]:
path = os.getcwd()+ '\..\Dados vetoriais\Pontos amostrais_Piracicaba\Dados_227pts_original_06_03_2021.shp'
pontos_pira_gpd = gpd.read_file(path)

In [ ]:
path2 = os.getcwd()+ '\..\Dados vetoriais\Piracicaba\Perimetro_Piracicaba.shp'
mapa_pira = gpd.read_file(path2)

In [ ]:
import json
from types import SimpleNamespace

mapa_pira["constant"] = 1
mapa_pira_dissolved = mapa_pira.dissolve(by = "constant").reset_index(drop = True)

asd = mapa_pira_dissolved.geometry
data = asd.to_json()

x = json.loads(data, object_hook=lambda d: SimpleNamespace(**d))
coordinates = x.features[0].geometry.coordinates
geometries = [item for sublist in coordinates for item in sublist]
geometries2 = [item for sublist in geometries for item in sublist]

geometries2.sort(key=lambda x:x[0])

In [ ]:
import numpy as np
from shapely.geometry import Point
from shapely.ops import unary_union
import geopandas as gpd

def generate_grid_in_polygon(spacing, polygon):
    ''' This Function generates evenly spaced points within the given GeoDataFrame.
        The parameter 'spacing' defines the distance between the points in coordinate units. '''
    
    # Convert the GeoDataFrame to a single polygon
    poly_in = unary_union([poly for poly in polygon.geometry])

    # Get the bounds of the polygon
    minx, miny, maxx, maxy = poly_in.bounds    
    
    # Now generate the entire grid
    x_coords = list(np.arange(np.floor(minx), int(np.ceil(maxx)), spacing))
    y_coords = list(np.arange(np.floor(miny), int(np.ceil(maxy)), spacing))
    
    grid = [Point(x) for x in zip(np.meshgrid(x_coords, y_coords)[0].flatten(), np.meshgrid(x_coords, y_coords)[1].flatten())]

    # Finally only keep the points within the polygon
    list_of_points = [point for point in grid if point.within(poly_in)]
    # list_of_points = [point for point in grid if poly_in.intersects(point)]

    return list_of_points

In [ ]:
points_in_poly = generate_grid_in_polygon(500, mapa_pira_dissolved)
points_in_poly_gdf = gpd.GeoDataFrame(geometry=points_in_poly)

pointList = [[point.x, point.y] for point in points_in_poly]

xgerados = [point.x for point in points_in_poly]
ygerados = [point.y for point in points_in_poly]

In [ ]:
fig, ax = plt.subplots(figsize = (50,50))
mapa_pira_dissolved.plot(ax=ax,facecolor='none', edgecolor='k')
points_in_poly_gdf.plot(ax=ax)

In [ ]:
B=regressor.predict(StdS_X.transform(pointList))

In [ ]:
dataFrameGerado = pd.DataFrame({'X':xgerados, 'Y': ygerados, 'Argila': B})

In [ ]:
mapa_gerado = gpd.GeoDataFrame(dataFrameGerado, geometry=gpd.points_from_xy(dataFrameGerado.X, dataFrameGerado.Y), crs = mapa_pira.crs)
mapa_gerado.plot()

In [ ]:
mapa_gerado["Argila"] = StdS_y.inverse_transform(B.reshape(-1,1))

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
mapa_gerado.plot("Argila",ax=ax, legend=True)

In [ ]:
# out_grid = make_geocube(vector_data=mapa_gerado, measurements=["Argila"], resolution=(-30,30),rasterize_function=rasterize_points_griddata)
# out_grid["Argila"].rio.to_raster("svr_out.tif")

In [ ]:
# src = rasterio.open("svr_out.tif")

# plt.imshow(src.read(1), cmap="pink")
# plt.show()
# src.close()